- OS에 ollama 설치 및 모델 다운로드 후 진행

In [ ]:
import requests
import time
import pandas as pd

OLLAMA_URL = "http://localhost:11434/api/generate"
MODELS = ["llama2", "llama3", "phi3"]
PROMPT_TEMPLATE = """
You are an ATC assistant AI system. Classify the pilot's request into one of the following categories:
- Bird risk
- Runway risk
- Other

Respond only with the category name.

Request: "{}"
"""

# 조종사 요청 샘플 불러오기
df = pd.read_csv("request_text_sample.csv")
queries = df["query"].tolist()

results = []

# LLM 호출 및 결과 저장
for query in queries:
    expected = "Bird risk"
    for model in MODELS:
        prompt = PROMPT_TEMPLATE.format(query)
        start = time.time()
        response = requests.post(OLLAMA_URL, json={"model": model, "prompt": prompt})
        end = time.time()

        if response.ok:
            output = response.json()["response"].strip()
        else:
            output = "ERROR"

        results.append({
            "query": query,
            "model": model,
            "output": output,
            "time": round(end - start, 3),
            "expected_intent": expected,
            "match": output.lower().strip() == expected
        })

# DataFrame으로 저장
df_result = pd.DataFrame(results)
df_result.to_csv("llm_result.csv", index=False)
print("저장 완료: llm_result.csv")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 결과 로드
df = pd.read_csv("llm_result.csv")

# 정확도 계산 및 시각화
accuracy_df = df.groupby("model")["match"].mean().reset_index()

plt.figure(figsize=(6, 4))
sns.barplot(data=accuracy_df, x="model", y="match")
plt.title("의도 분류 정확도")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.grid(True)
plt.show()

# 응답 시간 분포 시각화
plt.figure(figsize=(6, 4))
sns.boxplot(data=df, x="model", y="time")
plt.title("모델별 응답 시간 분포")
plt.ylabel("Time (seconds)")
plt.grid(True)
plt.show()

# 응답 일관성 계산
consistency_df = df.groupby("query")["output"].nunique().reset_index()
consistency_rate = (consistency_df["output"] == 1).mean()
print(f"응답 일관성 비율: {consistency_rate:.2f}")
